# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path

import numpy as np
import pandas as pd
import sys

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of notebook to set autoreload, and paths"""
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def colab_install_project_code(
    package_name:str # project package name, e.g. git+https://github.com/vtecftwy/metagentools.git@main
):
    """When nb is running on colab, pip install the project code package"""
    try:
        from google.colab import drive
        ON_COLAB = True
        print('The notebook is running on colab')
        print('Installing project code')
        cmd = f"pip install -U {package_name}"
        run(cmd)

    except ModuleNotFoundError:
        ON_COLAB = False
        print('The notebook is running locally, will not automatically install project code')

    return ON_COLAB

When using colab, the project code must be install every time from its github repo or from the Python Package Index (PyPI).

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
colab_install_project_code(package_name='git+https://github.com/vtecftwy/metagentools.git@main')

The notebook is running locally, will not automatically install project code


False

In [ ]:
#| export
def files_in_tree(
    path: str|Path,        # path to the directory to scan  
    pattern: str = ''      # pattern (glob style) to match in file name to filter the content
):
    """List files in directory and its subdiretories, print tree starting from parent directory"""
    parents = [p.name for p in path.parents]
    paths = []
    pad = ' ' * 2
    idx = 0
    print(f"{parents[0]}")
    print(f"{pad}|--{path.name}")
    for f in [p for p in path.glob(f"*{pattern}*") if p.is_file()]:
        paths.append(f)
        print(f"{pad}|{pad*2}|--{f.name} ({idx})")
        idx += 1
    for d in [p for p in path.glob(f"*{pattern}*") if p.is_dir()]:
        for f in d.iterdir():
            paths.append(f)
            print(f"{pad}|{pad*2}|--{f.name} ({idx})")
            idx += 1
    return paths

In [ ]:
p2dir = Path('').resolve()

files = files_in_tree(p2dir)
print(f"List of {len(files)} files when unfiltered")

ec-utils
  |--nbs-dev
  |    |--0_02_plotting.ipynb (0)
  |    |--2_01_image_utils.ipynb (1)
  |    |--1_01_eda_stats_utils.ipynb (2)
  |    |--0_01_ipython.ipynb (3)
  |    |--.last_checked (4)
  |    |--sidebar.yml (5)
  |    |--1_02_ml.ipynb (6)
  |    |--.ipynb_checkpoints (7)
  |    |--index.ipynb (8)
  |    |--nbdev.yml (9)
  |    |--9_01_dev_utils.ipynb (10)
  |    |--styles.css (11)
  |    |--_quarto.yml (12)
  |    |--0_02_plotting-checkpoint.ipynb (13)
  |    |--9_01_dev_utils-checkpoint.ipynb (14)
  |    |--0_01_ipython-checkpoint.ipynb (15)
  |    |--1_01_eda_stats_utils-checkpoint.ipynb (16)
  |    |--index-checkpoint.ipynb (17)
  |    |--2_01_image_utils-checkpoint.ipynb (18)
  |    |--1_02_ml-checkpoint.ipynb (19)
List of 20 files when unfiltered


In [ ]:
files = files_in_tree(p2dir, pattern='ipynb')
print(f"List of {len(files)} files when filtered")

ec-utils
  |--nbs-dev
  |    |--0_02_plotting.ipynb (0)
  |    |--2_01_image_utils.ipynb (1)
  |    |--1_01_eda_stats_utils.ipynb (2)
  |    |--0_01_ipython.ipynb (3)
  |    |--1_02_ml.ipynb (4)
  |    |--index.ipynb (5)
  |    |--9_01_dev_utils.ipynb (6)
  |    |--0_02_plotting-checkpoint.ipynb (7)
  |    |--9_01_dev_utils-checkpoint.ipynb (8)
  |    |--0_01_ipython-checkpoint.ipynb (9)
  |    |--1_01_eda_stats_utils-checkpoint.ipynb (10)
  |    |--index-checkpoint.ipynb (11)
  |    |--2_01_image_utils-checkpoint.ipynb (12)
  |    |--1_02_ml-checkpoint.ipynb (13)
List of 14 files when filtered


In [ ]:
#| export
def display_mds(
    *strings:str # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
           )

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several DataFrame in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-0.535612,-0.330359,-0.579999,1.004508,1.008913
1,-1.784759,0.157870,0.833984,-1.160063,-1.606875
2,0.843294,-0.924979,-1.127778,-1.590129,0.750082


,0,1,2,3,4,5,6,7,8,9
0,1.318010,0.020538,-0.937408,0.116248,-0.639289,-1.224406,-0.661524,-1.656005,0.744262,0.773764
1,-0.592513,0.898732,-0.723534,-1.010827,0.863045,0.766658,-0.767316,0.437185,0.635585,0.476885
2,-0.853805,1.772400,1.746037,1.272443,1.240098,1.313443,0.332629,-1.508551,0.317580,-1.013134


In [ ]:
#| hide
nbdev_export()